In [8]:
import numpy as np
import scipy.stats as st
from tqdm.notebook import tqdm

def coverage(band: np.ndarray, true_value: float) -> float:
    return np.mean(np.logical_and(band[0, :] <= true_value, band[1, :] >= true_value))

def cis(boot: np.ndarray, theta_hat: float, se_boot: float, alpha: float = 0.05):
    z = st.norm.ppf(1 - alpha/2)

    quantile_low = np.quantile(boot, alpha/2)
    quantile_top = np.quantile(boot, 1 - alpha/2)

    # Normal CI
    normal_ci = (theta_hat - z*se_boot, theta_hat + z*se_boot)

    # Percentile CI
    percentile_ci = (quantile_low, quantile_top)

    # Pivotal CI
    pivotal_ci = (2*theta_hat - quantile_top, 2*theta_hat - quantile_low)

    return normal_ci, percentile_ci, pivotal_ci

B = 1000
n = 25
n_exp = 1000

# True value
theta = (st.t.ppf(.75, df=3) - st.t.ppf(.25, df=3))/1.34

normal_band = np.zeros((2, n_exp))
percentile_band = np.zeros((2, n_exp))
pivot_band = np.zeros((2, n_exp))

for i in tqdm(range(n_exp)):
    X = st.t.rvs(3, size=n, random_state=i)

    theta_hat = (np.quantile(X, .75) - np.quantile(X, .25))/1.34

    xx = np.random.choice(X, size=(B,n), replace=True)
    boot: np.ndarray = ( np.quantile(xx, q=.75, axis=1) - np.quantile(xx, q=.25, axis=1) )/1.34
    se_boot = boot.std()

    # Confidence intervals
    all_cis = cis(boot, theta_hat, se_boot)

    normal_band[:, i] = all_cis[0]
    percentile_band[:, i] = all_cis[1]
    pivot_band[:, i] = all_cis[2]

ci_bands = [
    normal_band,
    percentile_band,
    pivot_band
]

for ci_band in ci_bands:
    print(coverage(ci_band, theta))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.95
0.967
0.827
